<div class="notebook-buttons" style="display:flex; padding-top: 5rem;padding-bottom: 2.5rem;line-height: 2.15;">
    <a href="https://colab.research.google.com/github/zazuko/notebooks/blob/master/notebooks/animal-disease/epidemics.ipynb">
        <div id="colab-link" style="display: flex;padding-right: 3.5rem;padding-bottom: 0.625rem;border-bottom: 1px solid #ececed; align-items: center;">
            <img class="call-to-action-img" src="img//colab.svg" width="30" height="30" style="margin-right: 10px;margin-top: auto;margin-bottom: auto;">
            <div class="call-to-action-txt">Run in Google Colab</div>
        </div>
    </a>
    <a href="https://raw.githubusercontent.com/zazuko/notebooks/master/notebooks/animal-disease/epidemics.ipynb" download>
        <div id="download-link" style="display: flex;padding-right: 3.5rem;padding-bottom: 0.625rem;border-bottom: 1px solid #ececed; height: auto;align-items: center;">
            <img class="call-to-action-img" src="img//download.svg" width="22" height="30" style="margin-right: 10px;margin-top: auto;margin-bottom: auto;">
            <div class="call-to-action-txt">Download Notebook</div>
        </div>
    </a>
    <a href="https://github.com/zazuko/notebooks/blob/master/notebooks/animal-disease/epidemics.ipynb">
        <div id="github-link" style="display: flex;padding-right: 3.5rem;padding-bottom: 0.625rem;border-bottom: 1px solid #ececed; height: auto;align-items: center;">
            <img class="call-to-action-img" src="img//github.svg" width="25" height="30" style="margin-right: 10px;margin-top: auto;margin-bottom: auto;">
            <div class="call-to-action-txt">View on GitHub</div>
        </div>
    </a>
</div>

# Animal epidemics in Switzerland

FOAG, Federal Office for Agriculture, collects data on the animal diseases in Switzerland. This data is published as [Linked Data](https://en.wikipedia.org/wiki/Linked_data). 

In this tutorial, we will show **how to work with Linked Data.** Mainly, we will see how to work with data on animal epidemics.   
We will look into how to query, process, and visualize it.   

## Ideas

* Animal hierarchy
* Disease hierarchy
* Number of reports per gde, over time (https://python-visualization.github.io/folium/plugins.html#folium.plugins.TimeSliderChoropleth)

* Serious outbreaks: find reports where stock == killed

In [ ]:
import json

import itertools
import folium
from folium.plugins import TimeSliderChoropleth
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime
import matplotlib.cm
import matplotlib as mpl
import mapclassify

from dateutil.relativedelta import relativedelta

from graphly.api_client import SparqlClient

import networkx as nx

from bokeh.io import show, output_notebook
from bokeh.plotting import from_networkx, ColumnDataSource

from bokeh.models import (Circle, EdgesAndLinkedNodes, HoverTool,
                          MultiLine, NodesAndLinkedEdges, Plot, TapTool,CustomJS, LabelSet)
from bokeh.palettes import Paired

import pandas as pd

In [ ]:
sparql = SparqlClient("https://int.lindas.admin.ch/query")
geosparql = SparqlClient("https://ld.geo.admin.ch/query")

sparql.add_prefixes({
    "schema": "<http://schema.org/>",
    "cube": "<https://cube.link/>",
    "admin": "<https://schema.ld.admin.ch/>",
    "skos": "<http://www.w3.org/2004/02/skos/core#>",
    "disease": "<https://environment.ld.admin.ch/foen/animal-pest/>"   
})

geosparql.add_prefixes({
    "dct": "<http://purl.org/dc/terms/>",
    "geonames": "<http://www.geonames.org/ontology#>",
    "schema": "<http://schema.org/>",
    "geosparql": "<http://www.opengis.net/ont/geosparql#>",
})

### Animals hierarchy

In [ ]:
query = """
SELECT DISTINCT ?specie ?group
WHERE {
  ?s disease:animal-specie ?specieIRI.
  ?specieIRI schema:name ?specie.
  
  ?specieIRI skos:broader/schema:name ?group.
  
  FILTER (LANG(?specie) = "de")
  FILTER (LANG(?group) = "de")
} 
ORDER BY ?group
"""

df = sparql.send_query(query)
df.head()

## Disease hierarchy

In [ ]:
query = """
SELECT DISTINCT ?epidemics ?group
WHERE {
  ?s disease:epidemics ?epidemicsIRI.
  ?epidemicsIRI schema:name ?epidemics.
  
  ?epidemicsIRI skos:broader/schema:name ?group.

  FILTER (LANG(?epidemics) = "de")
  FILTER (LANG(?group) = "de")
}
ORDER BY ?group
"""

df = sparql.send_query(query)
df.head()

## Can we link disease to animal type?

In [ ]:
query = """
SELECT DISTINCT ?epidemics ?specie ?group
WHERE {
  <https://environment.ld.admin.ch/foen/animal-pest/observation/> cube:observation ?obs .
  ?obs disease:epidemics/schema:name ?epidemics;
       disease:animal-specie ?specieIRI.
       
  ?specieIRI schema:name ?specie.
  ?specieIRI skos:broader/schema:name ?group.
  
  FILTER (LANG(?epidemics) = "de")
  FILTER (LANG(?specie) = "de")
  FILTER (LANG(?group) = "de")
} 

ORDER BY ?specie
"""

df = sparql.send_query(query)
df.head()

In [ ]:
g = nx.from_pandas_edgelist(df, source='epidemics', target='specie')
groups = {key: "epidemics" for key in df.epidemics} | {key: "specie" for key in df.specie}

nx.set_node_attributes(g, groups, name="group")

colormap = {"epidemics": Paired[4][0], "specie": Paired[4][2]}
colors_fill = {k: colormap[v] for k, v in groups.items()}

colormap = {"epidemics": Paired[4][1], "specie": Paired[4][3]}
colors_hover = {k: colormap[v] for k, v in groups.items()}

nx.set_node_attributes(g, colors_fill, name="fill_color")
nx.set_node_attributes(g, colors_hover, name="hover_color")

animals = df.specie.unique()
epidemics = df.epidemics.unique()

In [ ]:
# Loayouts https://networkx.org/documentation/stable/reference/generated/networkx.drawing.layout.spring_layout.html
plot = Plot(plot_width = 1000, plot_height=1000)
plot.title.text = 'Bokeh Plot'

graph_renderer = from_networkx(g, nx.drawing.layout.bipartite_layout, nodes = animals)

# manipulating nodes
graph_renderer.node_renderer.glyph = Circle(size = 15, fill_color = "fill_color")
graph_renderer.node_renderer.nonselection_glyph = Circle(size = 15, fill_color = "fill_color")
graph_renderer.node_renderer.selection_glyph = Circle(size = 15, fill_color = "hover_color")
graph_renderer.node_renderer.hover_glyph = Circle(size = 15, fill_color = "hover_color")
graph_renderer.node_renderer.data_source.data['group'] = [groups[g] for g in graph_renderer.node_renderer.data_source.data["index"]]

# manipulating edges
graph_renderer.edge_renderer.glyph = MultiLine(line_color = '#CCCCCC', line_alpha = 
 .5, line_width = 1)
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color = Paired[7][-1], line_width = 2)
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color = Paired[7][-1], line_width = 2)


graph_renderer.selection_policy = NodesAndLinkedEdges()
graph_renderer.inspection_policy = NodesAndLinkedEdges()

plot.renderers.append(graph_renderer)

code = '''
    if (cb_data.index.indices.length > 0) {
        const index = cb_data.index.indices[0];

        if (source.data.group[index] === "specie") {
            hover.tooltips = [["Animal", "@index"]];  
        }
        else {
            hover.tooltips = [["Disease", "@index"]];
        }                                     
    }
'''

hover = HoverTool(
    renderers=[graph_renderer.node_renderer]
)
hover.callback = CustomJS(
    args = dict(source = graph_renderer.node_renderer.data_source, hover = hover),
    code = code
)

pos = {k: v for k, v in graph_renderer.layout_provider.graph_layout.items() if g.nodes[k]["group"] == "specie"}
names = list(pos.keys())
x, y = map(list, zip(*pos.values()))

source = ColumnDataSource({'x': x, 'y': y, 'name': names})
labels = LabelSet(x_offset=15, y_offset=-6, x='x', y='y', text='name', source=source, background_fill_color='white', text_font_size='10px', background_fill_alpha=.7)
plot.renderers.append(labels)

plot.add_tools(hover, TapTool())

output_notebook()
show(plot)

### Diseases
All reports. Run in browser: https://s.zazuko.com/24fSKE

In [ ]:
query = """
SELECT ?diagnosis ?gde ?specie ?stock ?sick ?infected ?killed ?deceased ?epidemics ?death_cause
WHERE {
  <https://environment.ld.admin.ch/foen/animal-pest/observation/> cube:observation ?obs .
  ?obs disease:epidemics/schema:name ?epidemics;
       disease:diagnosis-date ?diagnosis;
       disease:animals-stock ?stock;
       disease:animals-sick ?sick;
       disease:animals-infected ?infected;
       disease:animals-killed ?killed;
       disease:animals-deceased ?deceased;
       disease:internet-publication ?date;
       disease:death-cause/schema:name ?death_cause;
       disease:animal-specie/schema:name ?specie;
       schema:containedInPlace/schema:name ?gde .
  
  FILTER (LANG(?epidemics) = "de")
  FILTER (LANG(?death_cause) = "de")
  FILTER (LANG(?specie) = "de")
} 
ORDER BY DESC(?diagnosis) ?gde
"""
df = sparql.send_query(query)
df.head()

## Reports

Number of reports per day, specie, commune, and epidemics

In [ ]:
query = """
SELECT ?diagnosis ?municipality_id ?gde ?specie ?epidemics
WHERE {
  <https://environment.ld.admin.ch/foen/animal-pest/observation/> cube:observation ?obs .
  ?obs disease:epidemics/schema:name ?epidemics;
       disease:diagnosis-date ?diagnosis;
       disease:internet-publication ?date;
       disease:animal-specie/schema:name ?specie;
       schema:containedInPlace ?municipality_id.
       
  ?municipality_id schema:name ?gde .
  
  FILTER (LANG(?epidemics) = "de")
  FILTER (LANG(?specie) = "de")
} 
ORDER BY DESC(?diagnosis) ?gde
"""
df = sparql.send_query(query)

In [ ]:
df.loc[:,"year"] = df.diagnosis.apply(lambda x: x.year).astype(int)
df.loc[:,"month"] = df.diagnosis.apply(lambda x: x.month)
df.loc[:,"week"] = df.diagnosis.apply(lambda x: x.isocalendar()[1])
df["municipality_id"] = df.municipality_id.str.split("/").str[-1]

weekly = True
if weekly:
    df["diagnosis"] = df.diagnosis.apply(lambda x: datetime.date(x.year, 1, 1) + relativedelta(weeks=+x.isocalendar()[1]))
else:
    df['diagnosis'] = df.diagnosis.apply(lambda x: datetime.date(x.year, x.month, 1))

df = df[["diagnosis", "specie", "epidemics", "municipality_id"]].groupby(by=["diagnosis", "specie", "epidemics", "municipality_id"]).size().reset_index(name="reports")
df.head()

In [ ]:
dff = df.copy()
dff = dff[["diagnosis", "specie", "epidemics"]].groupby(by=["diagnosis", "specie", "epidemics"]).size().reset_index(name="reports")
dff = dff.sort_values(by=["diagnosis", "epidemics", "specie"], ascending=False)
dff.sort_values(by=["reports"], ascending=False).reset_index(drop=True)[0:15]

## All epidemics for one animal type

In [ ]:
MIN_TOTAL = 30
MIN_LOCAL = 5
SPECIE = "Ziege"

subset = dff[dff.specie == SPECIE]
diseases = subset[["epidemics", "reports"]].groupby(["epidemics"]).agg(["sum", "max"]).reset_index()
relevant_diseases = list(diseases[(diseases["reports"]["sum"] > MIN_TOTAL) & (diseases["reports"]["max"] > MIN_LOCAL)].epidemics)
subset = subset[subset.epidemics.isin(relevant_diseases)]

# TODO: all dates -> all possible dates
all_dates = subset.diagnosis.unique()
combinations = list(itertools.product(*[all_dates, subset.epidemics.unique()]))
diseases_empty = pd.DataFrame(combinations, columns =['diagnosis', 'epidemics'])
subset = pd.merge(diseases_empty, subset, how="left", on=["diagnosis", "epidemics"]).fillna(0)

fig = go.Figure()
for curr_disease in subset.epidemics.unique():
    plot_df = subset[subset.epidemics == curr_disease]
    labels = ["<b>{}</b><br>Date: {}<br>Reports: {}".format(curr_disease, row.diagnosis, int(row.reports)) for row in plot_df[["diagnosis", "reports"]].itertuples()]
    fig.add_trace(go.Scatter(x=plot_df.diagnosis, y=plot_df.reports, name=curr_disease,
                        text=labels,
                        hoverinfo='text',
                        line_shape='spline'))

fig.update_xaxes(rangeslider_visible=True)
fig.show()

## Selected epidemics

In [ ]:
d = dff[dff.epidemics == "Bovine Virusdiarrhoe / Mucosal Disease"]
fig = px.line(d, x="diagnosis", y="reports")
fig.show()

In [ ]:
d = dff[dff.epidemics == "Sauerbrut der Bienen"]
fig = px.bar(d, x="diagnosis", y="reports")
fig.update_xaxes(rangeslider_visible=True)
fig.show()

Interesting case: rabies does not appear anymore!

Also, check rabies by specie: all cases were coming from one animal type: Fuchs

In [ ]:
d = dff[dff.epidemics == "Tollwut"]
fig = px.bar(d, x="diagnosis", y="reports")
fig.show()

## Biggest epidemics

In [ ]:
diseases = dff[["reports", "epidemics"]].groupby("epidemics").sum().reset_index().sort_values(by="reports", ascending=False).reset_index(drop=True)
ROWS=8
COLS=2

diseases_subset = diseases[0:ROWS*COLS]
fig = make_subplots(rows=ROWS, cols=COLS, subplot_titles=diseases_subset["epidemics"])
for obs in diseases_subset.itertuples():
    
    row = obs.Index//COLS + 1
    col = obs.Index%COLS + 1
    d = dff[dff.epidemics == obs.epidemics]
    fig.append_trace(go.Line(x=d["diagnosis"], y=d["reports"], name=obs.epidemics, marker_color=px.colors.qualitative.Dark24[0]), row=row, col=col)
    
fig.update_layout(height=1200, width=1000, title={"text": "Biggest animal epidemics in Switzerland", "x": 0.5}, showlegend=False)
fig.show()

In [ ]:
diseases = dff[["reports", "epidemics", "specie"]].groupby(["specie", "epidemics"]).sum().reset_index()
diseases["reports_group"] = diseases[["epidemics", "reports"]].groupby("epidemics").transform(sum)
diseases["affected_species"] = diseases[["epidemics", "reports"]].groupby("epidemics").transform(len)
diseases = diseases[diseases.affected_species > 1]
diseases = diseases.sort_values(by=["reports_group", "reports"], ascending=False).reset_index(drop=True)
diseases[diseases.epidemics != "Salmonellose"][0:10]

## Multiple species affected by one disease

## Disease co-ocurrence: Chlamydienabort
https://www.blv.admin.ch/blv/fr/home/tiere/tierseuchen/uebersicht-seuchen/alle-tierseuchen/chlamydienabort-der-schafe-und-ziegen.html

In [ ]:
# Candidates: Coxiellose, Campylobacteriose, Listeriose, Pseudotuberkulose der Schafe und Ziegen
# Interesting :Tollwut
# SUPER_ CURIOUS: Maedi-Visna

#epidemic = "Chlamydienabort der Schafe und Ziegen"
epidemic = "Maedi-Visna"
#epidemic = "Pseudotuberkulose der Schafe und Ziegen"
relevant_animals = diseases[(diseases.epidemics == epidemic) & (diseases.reports > 20)].specie
subset = dff[(dff.epidemics == epidemic) & (dff.specie.isin(relevant_animals))]

fig = px.line(subset, x="diagnosis", y="reports", color="specie", range_y=[0, int(max(subset.reports)*1.3)+1], range_x=[min(subset.diagnosis), max(subset.diagnosis)])
        
fig.update_xaxes(rangeslider_visible=True)
fig.show()

### Epidemics over time

In [ ]:
query = """
SELECT ?municipality_id ?municipality ?boundary 

WHERE {
  ?muni_iri dct:hasVersion ?version ;
            geonames:featureCode geonames:A.ADM3 .
  
  ?version schema:validUntil "2020-12-31"^^<http://www.w3.org/2001/XMLSchema#date>;
           schema:name ?municipality;
           geosparql:hasGeometry/geosparql:asWKT ?boundary.
  
  BIND(IRI(REPLACE(STR(?muni_iri), "https://ld.geo.admin.ch/boundaries/", "https://ld.admin.ch/")) AS ?municipality_id)
}
"""
communes = geosparql.send_query(query)
communes = communes.set_crs(epsg=4326)

In [ ]:
communes["municipality_id"] = communes.municipality_id.str.split("/").str[-1]
communes = communes.set_index("municipality_id")
communes.head()

In [ ]:
sickness = "Bovine Virusdiarrhoe / Mucosal Disease"
specie = "Rind"
start = datetime.date(2007, 9, 1)
end = datetime.date(2010, 3, 1)
WINDOW = 5

subset = df[(df.specie == specie) & (df.epidemics == sickness) & (df.diagnosis >= start) & (df.diagnosis <= end)].copy()
s = subset[["diagnosis", "municipality_id", "reports"]].pivot(index="diagnosis", columns="municipality_id", values="reports").sort_index().fillna(0)
s = s.rolling(WINDOW).sum().iloc[WINDOW-1:,:]

datetime_index = pd.DatetimeIndex(s.index)
dt_index_epochs = datetime_index.view("int") // 10**9
s.index = dt_index_epochs.astype('U10')

In [ ]:
bins = [1,2,5,10,25]
#classifier = mapclassify.UserDefined(y=s.values[s.values > 0], bins=bins)
classifier = mapclassify.NaturalBreaks(y=s.values[s.values > 0])
categories = range(len(bins))
colormap = {cat: mpl.colors.rgb2hex(mpl.cm.PuRd(i/(len(categories)-1))) for i, cat in enumerate(categories)}

styledict = {}
timestamps = s.index
for commune in communes.index:
    styledict[commune] = {str(t): {"color": colormap[0], "opacity": 1} for t in timestamps}
    
    if commune in s.columns:
        for timestamp in timestamps:
            reports = s.loc[timestamp, commune]

            #if reports > 0:
            reports_classified = classifier(reports)[0]
            styledict[commune][str(timestamp)] = {"color": colormap[reports_classified], "opacity": 1}#"red"


#bins = list(classifier.bins)
communes["dummy"] = 0
m = folium.Map(location=[46.83, 8.13], zoom_start=8, tiles="cartodbpositron")

g = TimeSliderChoropleth(
    communes.to_json(),
    styledict=styledict
).add_to(m)

folium.Choropleth(
    geo_data=json.loads(communes.to_json()),
    data=communes,
    columns=['municipality', 'dummy'],
    key_on='feature.id',
    fill_color= 'PuRd',
    fill_opacity=0.0,
    line_opacity=0.0,
    bins=[0]+bins,
    legend_name="Cases of {}".format(sickness)
).add_to(m)

folium.LayerControl().add_to(m)
m

### Deadly epidemics

Which epidemics lead to death of all animals?

In [ ]:
query = """
SELECT ?diagnosis ?gde ?specie ?stock_deceased ?epidemics ?death_cause
WHERE {
  <https://environment.ld.admin.ch/foen/animal-pest/observation/> cube:observation ?obs .
  ?obs disease:epidemics/schema:name ?epidemics;
       disease:diagnosis-date ?diagnosis;
       disease:animals-stock ?stock_deceased;
       disease:animals-deceased ?stock_deceased;
       disease:internet-publication ?date;
       disease:death-cause/schema:name ?death_cause;
       disease:animal-specie/schema:name ?specie;
       schema:containedInPlace/schema:name ?gde .
  
  #FILTER (?stock_deceased > 1)
  FILTER (LANG(?epidemics) = "de")
  FILTER (LANG(?death_cause) = "de")
  FILTER (LANG(?specie) = "de")
} 
ORDER BY DESC(?stock_deceased)
"""

#df = sparql.send_query(query)
#df.head()

A disease can be higly contagious. It may be hence sefer to kill all animal stock at certain farm, and prevent potential disease spread.

Which epidemics forced farmers to kill all their stock?

In [ ]:
query = """
SELECT ?diagnosis ?gde ?specie ?sick ?infected ?deceased ?stock_killed ?epidemics ?death_cause
WHERE {
  <https://environment.ld.admin.ch/foen/animal-pest/observation/> cube:observation ?obs .
  ?obs disease:epidemics/schema:name ?epidemics;
       disease:diagnosis-date ?diagnosis;
       disease:animals-stock ?stock_killed;
       disease:animals-sick ?sick;
       disease:animals-infected ?infected;
       disease:animals-killed ?stock_killed;
       disease:animals-deceased ?deceased;
       disease:internet-publication ?date;
       disease:death-cause/schema:name ?death_cause;
       disease:animal-specie/schema:name ?specie;
       schema:containedInPlace/schema:name ?gde .
  
  #FILTER (?stock_killed > 1)
  FILTER (LANG(?epidemics) = "de")
  FILTER (LANG(?death_cause) = "de")
  FILTER (LANG(?specie) = "de")
} 
ORDER BY DESC(?stock_killed)
"""

#df = sparql.send_query(query)
#df.head()